In [1]:
import torch 
import pandas as pd 
import numpy as np
import os
from lincs_gsnn.data.DXDTDataset import DXDTDataset
from torch.utils.data import DataLoader 

from matplotlib import pyplot as plt
from sklearn.metrics import r2_score

import networkx as nx 

In [7]:
data = torch.load('/home/teddy/local/lincs-gsnn/workflow/runs/default/bionetwork/bionetwork.pt', weights_only=False)

In [11]:
drug = 'BRD-K33379087'

In [12]:
inp_names   = data['node_names_dict']['input']
func_names  = data['node_names_dict']['function']
edge_mat    = data['edge_index_dict'][('input','to','function')]

try:
    drug_idx = inp_names.index(f"DRUG__{drug}")
except ValueError:
    raise KeyError(f"{drug} not found among {len(inp_names)} input nodes")

src, dst         = edge_mat
mask             = (src == drug_idx)
drug_targets_idx = dst[mask].cpu().tolist()
drug_targets     = [func_names[i] for i in drug_targets_idx]

In [13]:
drug_targets

['PROTEIN__P08581']

In [15]:
tge = pd.read_csv('../../data/targetome_extended-01-23-25.csv')
druginfo = pd.read_csv('../../data/compoundinfo_beta.txt', sep='\t')
tge = tge.merge(druginfo[['inchi_key', 'pert_id']].drop_duplicates(), on='inchi_key')

tge = tge[lambda x: x.pert_id == drug]
tge = tge[lambda x: x.assay_type.isin(['Kd', 'Ki']) & (x.assay_relation.isin(['<', '<=', '=']) & (x.assay_value < 1000))][['pert_id','uniprot_id']].drop_duplicates()
tge.head()

,pert_id,uniprot_id
119923,BRD-K33379087,O60313
119924,BRD-K33379087,O60841
119958,BRD-K33379087,P08581
120003,BRD-K33379087,P30519
